**Question 3**

In [16]:
from scipy.optimize import linprog

def find_decomposition(budget: list[float], preferences:list [set[int]]):
    """
    Check if a budget is decomposable:
    Each citizen gives their fair share only to topics they support,
    and all topics receive exactly what they need.
    Returns the donation matrix if possible, else None.

    Programmer: Vivian Umansky
    Date: June 2025
    """
    num_citizens = len(preferences)
    num_topics = len(budget)
    total_budget = sum(budget)
    citizen_budget = total_budget / num_citizens

    # We define one variable per (citizen, topic) pair.
    num_vars = num_citizens * num_topics
    objective = [0] * num_vars

    A_eq = []
    b_eq = []

    # Constraint: Each citizen donates exactly their fair share
    for i in range(num_citizens):
        row = [0] * num_vars
        for j in range(num_topics):
            row[i * num_topics + j] = 1
        A_eq.append(row)
        b_eq.append(citizen_budget)

    # Constraint: Each topic receives the exact amount required
    for j in range(num_topics):
        row = [0] * num_vars
        for i in range(num_citizens):
            row[i * num_topics + j] = 1
        A_eq.append(row)
        b_eq.append(budget[j])

    # Bounds: Citizens can donate only to supported topics
    bounds = []
    for i in range(num_citizens):
        for j in range(num_topics):
            if j in preferences[i]:
                bounds.append((0, None))  # Allowed to donate
            else:
                bounds.append((0, 0)) # Must be zero

    result = linprog(c=objective, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

    if result.success:
      solution = result.x
      donations = []
      for i in range(num_citizens):
          row = []
          for j in range(num_topics):
              # Round and convert
              value = round(solution[i * num_topics + j], 2)
              row.append(float(value))
          donations.append(row)
      return donations
    else:
            return None

# -------------------------------------------
# Example
# -------------------------------------------
def main():
    examples = [
        {
            "title": "Example 1 — from the question",
            "budget": [400, 50, 50, 0],
            "preferences": [
                {0, 1},
                {0, 2},
                {0, 3},
                {1, 2},
                {0}
            ]
        },
        {
            "title": "Example 2 — balanced between two topics",
            "budget": [150, 150, 0],
            "preferences": [
                {0, 1},
                {0},
                {1},
                {0, 1}
            ]
        }
    ]

    for example in examples:
        print(f"\n{'='*60}\n{example['title']}\n{'='*60}")
        budget = example["budget"]
        preferences = example["preferences"]

        result = find_decomposition(budget, preferences)

        if result:
            num_citizens = len(result)
            num_topics = len(budget)
            total_budget = sum(budget)
            share = total_budget / num_citizens

            print("The budget is decomposable. One possible decomposition:\n")

            # Table header
            header = "Citizen | " + " | ".join([f"Topic {j}" for j in range(num_topics)]) + " | Total"
            print(header)
            print("-" * len(header))

            # Print each citizen's donations
            for i, row in enumerate(result):
                row_sum = sum(row)
                row_str = "   {:<4} | ".format(i) + " | ".join(f"{x:7.2f}" for x in row) + f" | {row_sum:.2f}"
                print(row_str)

            # Print how much each topic received
            print("\nTotal received per topic:")
            for j in range(num_topics):
                topic_total = sum(result[i][j] for i in range(num_citizens))
                print(f"Topic {j}: received {topic_total:.2f} (expected: {budget[j]})")

            # Check for illegal donations
            print("\nChecking that citizens donate only to supported topics:")
            for i in range(num_citizens):
                for j in range(num_topics):
                    if result[i][j] > 0 and j not in preferences[i]:
                        print(f"Citizen {i} donated to topic {j} without supporting it!")
            print("All donations match citizen preferences.")

            # Check each citizen's total donation
            print("\nVerifying citizen donation amounts:")
            for i, row in enumerate(result):
                total = sum(row)
                print(f"Citizen {i} donated {total:.2f} (expected: {share:.2f})")

            print("\nAll constraints are satisfied.")
        else:
            print("The budget is not decomposable")

if __name__ == "__main__":
    main()


Example 1 — from the question
The budget is decomposable. One possible decomposition:

Citizen | Topic 0 | Topic 1 | Topic 2 | Topic 3 | Total
-------------------------------------------------------
   0    |  100.00 |    0.00 |    0.00 |    0.00 | 100.00
   1    |  100.00 |    0.00 |   -0.00 |    0.00 | 100.00
   2    |  100.00 |    0.00 |    0.00 |    0.00 | 100.00
   3    |    0.00 |   50.00 |   50.00 |    0.00 | 100.00
   4    |  100.00 |    0.00 |    0.00 |    0.00 | 100.00

Total received per topic:
Topic 0: received 400.00 (expected: 400)
Topic 1: received 50.00 (expected: 50)
Topic 2: received 50.00 (expected: 50)
Topic 3: received 0.00 (expected: 0)

Checking that citizens donate only to supported topics:
All donations match citizen preferences.

Verifying citizen donation amounts:
Citizen 0 donated 100.00 (expected: 100.00)
Citizen 1 donated 100.00 (expected: 100.00)
Citizen 2 donated 100.00 (expected: 100.00)
Citizen 3 donated 100.00 (expected: 100.00)
Citizen 4 donated 100